# **A program For Photographers Event**

## **Introduction**

Nepal is a beautiful country with a great prospect of tourism. Every year a lot of tourists come to visit Nepal. Hence in this analytical project I want find the best region of Kathmandu for tourist to stay such that they have maximum access to facilities.

## **Data Description**

The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.

## **Table of contents**

#### 1- Import Libraries
#### 2- Define Foursquare Credentials
#### 3- Search for Hotels
#### 4- Search for Temples
#### 5- Search for Resturants
#### 6- Search for Cafe
#### 7- Search for Shopping Mall
#### 8- Generating Map for Analysis

## **Import Libraries**

In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

## **Define Foursquare Credentials**

##### code has been removed

## **Define the city and get its latitude & longitude** 

In [9]:
# define the city and get its latitude & longitude 
city = 'Kathmandu'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

27.708796 85.320244


## **Search for Hotels**

In [ ]:
# search for hotels
search_query = 'Hotel'
radius = 20000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [69]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [70]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,4d6904f9b6f46dcbc01c1cb2,Nachghar,NP,काठमाडौं,नेपाल,Jamal,364,"[Nachghar (Jamal), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.7100008, 'lng'...",27.710001,85.316806,Central Region,Green Party Palace,v-1562690342
1,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4d74e17ae278f04d6d225db8,kamaladi,NP,काठमाडौं,नेपाल,NaN,351,"[kamaladi, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.7067524, 'lng'...",27.706752,85.322962,Central Region,Jai Ganesh Kamaladi Party Palace,v-1562690342
2,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",False,50ba2197498e775084c60691,NaN,NP,NaN,नेपाल,NaN,782,[नेपाल],"[{'label': 'display', 'lat': 27.70970633844538...",27.709706,85.328112,NaN,Celebration Party Venue,v-1562690342
3,[],False,4d666a4648ee6ea829046568,NaN,NP,NaN,नेपाल,NaN,760,[नेपाल],"[{'label': 'display', 'lat': 27.714899, 'lng':...",27.714899,85.323715,NaN,Lainchaur Party Venue,v-1562690342
4,"[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",False,4d611aae196ba093a5cc2156,Tredevi Marg,NP,काठमाडौं,नेपाल,NaN,844,"[Tredevi Marg, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71399812380037...",27.713998,85.314003,Central Region,New Tridevi Party Venue,v-1562690342


## **Clean Hotel Dataframe**

In [71]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Green Party Palace,Asian Restaurant,Nachghar,NP,काठमाडौं,नेपाल,Jamal,364,"[Nachghar (Jamal), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.7100008, 'lng'...",27.710001,85.316806,Central Region,4d6904f9b6f46dcbc01c1cb2
1,Jai Ganesh Kamaladi Party Palace,Food Court,kamaladi,NP,काठमाडौं,नेपाल,NaN,351,"[kamaladi, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.7067524, 'lng'...",27.706752,85.322962,Central Region,4d74e17ae278f04d6d225db8
2,Celebration Party Venue,Event Space,NaN,NP,NaN,नेपाल,NaN,782,[नेपाल],"[{'label': 'display', 'lat': 27.70970633844538...",27.709706,85.328112,NaN,50ba2197498e775084c60691
3,Lainchaur Party Venue,None,NaN,NP,NaN,नेपाल,NaN,760,[नेपाल],"[{'label': 'display', 'lat': 27.714899, 'lng':...",27.714899,85.323715,NaN,4d666a4648ee6ea829046568
4,New Tridevi Party Venue,Other Nightlife,Tredevi Marg,NP,काठमाडौं,नेपाल,NaN,844,"[Tredevi Marg, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71399812380037...",27.713998,85.314003,Central Region,4d611aae196ba093a5cc2156


In [72]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'postalCode', 'id'], axis=1)
clean_dataframe2.head()

KeyError: "['neighborhood' 'postalCode'] not found in axis"

In [15]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3.head()

,name,categories,address,lat,lng,state
0,Hotel Yak & Yeti,Hotel,Lalupate Marg,27.711581,85.320274,Central Region
1,Hotel Shanker,Hotel,Lazimpat,27.718956,85.320082,Central Region
2,Royal Singhi Hotel,Hotel,Lal Durbar,27.710940,85.319466,Central Region
3,De L'Annapurna Hotel,Hotel,Durbar Marg,27.711117,85.316408,Central Region
4,Hotel Buddha,Hotel,25728,27.717044,85.310450,Kathmando


In [16]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space', 'Resort']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe = clean_dataframe3

In [17]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels.head()

,name,categories,address,lat,lng,state
0,Hotel Yak & Yeti,Hotel,Lalupate Marg,27.711581,85.320274,Central Region
1,Hotel Shanker,Hotel,Lazimpat,27.718956,85.320082,Central Region
2,Royal Singhi Hotel,Hotel,Lal Durbar,27.710940,85.319466,Central Region
3,De L'Annapurna Hotel,Hotel,Durbar Marg,27.711117,85.316408,Central Region
4,Hotel Buddha,Hotel,25728,27.717044,85.310450,Kathmando


## **Search for temples**

In [50]:
# search for temples
search_query = 'Temple'
radius = 20000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [21]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [22]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
temples_dataframe = json_normalize(venues)
temples_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d13a941735', 'name': 'T...",False,4bcf47520ffdce724457b2c0,Chusyabahal,NP,काठमाडौं,नेपाल,Jyatha,769,"[Chusyabahal (Jyatha), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71123533061801...",27.711235,85.312934,NaN,Central Region,Kantipur Temple House,v-1562688682
1,"[{'id': '52e81612bcbc57f1066b7a3e', 'name': 'B...",False,51790da9e4b0de302b37fed6,Kwabahal,NP,Pātan,नेपाल,NaN,3757,"[Kwabahal, Patan, नेपाल]","[{'label': 'display', 'lat': 27.67524821814495...",27.675248,85.324419,NaN,Central Region,Hiranya Varna Mahavihar (Golden Temple),v-1562688682
2,"[{'id': '4bf58dd8d48988d13a941735', 'name': 'T...",False,527b6c6511d28e4aa2a056f9,NaN,NP,NaN,नेपाल,NaN,555,[नेपाल],"[{'label': 'display', 'lat': 27.70881275112659...",27.708813,85.314609,NaN,NaN,Bal Gopal Temple,v-1562688682
3,"[{'id': '4bf58dd8d48988d13a941735', 'name': 'T...",False,5690fc4338fafe86458fef24,NaN,NP,NaN,नेपाल,NaN,1096,[नेपाल],"[{'label': 'display', 'lat': 27.705959, 'lng':...",27.705959,85.309586,NaN,NaN,Akash Bhairab Temple,v-1562688682
4,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",False,507e919ee4b00e30b6288444,NaN,NP,NaN,नेपाल,NaN,701,[नेपाल],"[{'label': 'display', 'lat': 27.71498960118585...",27.714990,85.321530,NaN,NaN,Golden Temple,v-1562688682


## **Clean temples Dataframe**

In [23]:
# keep only columns that include venue name, and anything that is associated with location
temples_clean_columns = ['name', 'categories'] + [col for col in temples_dataframe.columns if col.startswith('location.')]+ ['id']
clean_temples_dataframe = temples_dataframe.loc[:,temples_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_temples_dataframe['categories'] = clean_temples_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_temples_dataframe.columns = [column.split('.')[-1] for column in clean_temples_dataframe.columns]

clean_temples_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Kantipur Temple House,Temple,Chusyabahal,NP,काठमाडौं,नेपाल,Jyatha,769,"[Chusyabahal (Jyatha), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71123533061801...",27.711235,85.312934,NaN,Central Region,4bcf47520ffdce724457b2c0
1,Hiranya Varna Mahavihar (Golden Temple),Buddhist Temple,Kwabahal,NP,Pātan,नेपाल,NaN,3757,"[Kwabahal, Patan, नेपाल]","[{'label': 'display', 'lat': 27.67524821814495...",27.675248,85.324419,NaN,Central Region,51790da9e4b0de302b37fed6
2,Bal Gopal Temple,Temple,NaN,NP,NaN,नेपाल,NaN,555,[नेपाल],"[{'label': 'display', 'lat': 27.70881275112659...",27.708813,85.314609,NaN,NaN,527b6c6511d28e4aa2a056f9
3,Akash Bhairab Temple,Temple,NaN,NP,NaN,नेपाल,NaN,1096,[नेपाल],"[{'label': 'display', 'lat': 27.705959, 'lng':...",27.705959,85.309586,NaN,NaN,5690fc4338fafe86458fef24
4,Golden Temple,Historic Site,NaN,NP,NaN,नेपाल,NaN,701,[नेपाल],"[{'label': 'display', 'lat': 27.71498960118585...",27.714990,85.321530,NaN,NaN,507e919ee4b00e30b6288444


In [24]:
# delete unnecessary columns
clean_temples_dataframe2= clean_temples_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','crossStreet','postalCode', 'id'], axis=1)
clean_temples_dataframe2.head()

,name,categories,address,lat,lng,state
0,Kantipur Temple House,Temple,Chusyabahal,27.711235,85.312934,Central Region
1,Hiranya Varna Mahavihar (Golden Temple),Buddhist Temple,Kwabahal,27.675248,85.324419,Central Region
2,Bal Gopal Temple,Temple,NaN,27.708813,85.314609,NaN
3,Akash Bhairab Temple,Temple,NaN,27.705959,85.309586,NaN
4,Golden Temple,Historic Site,NaN,27.714990,85.321530,NaN


In [25]:
# delete rows with none values
clean_temples_dataframe3 = clean_temples_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_temples_dataframe3.head()

,name,categories,address,lat,lng,state
0,Kantipur Temple House,Temple,Chusyabahal,27.711235,85.312934,Central Region
1,Hiranya Varna Mahavihar (Golden Temple),Buddhist Temple,Kwabahal,27.675248,85.324419,Central Region
5,The Temple,Bar,Thamel,27.715492,85.310461,Central Region
6,Pashupatinath Temple,Temple,Pashupatinath Rd.,27.709101,85.348620,Central Region
8,Taleju Temple,Temple,Makhan Tole,27.712171,85.311342,Central Region


In [63]:
# delete rows which its category is not temples
df_temples = clean_temples_dataframe3[clean_temples_dataframe3.categories != 'Bar']
df_temples.head()

,name,categories,address,lat,lng,state
0,Kantipur Temple House,Temple,Chusyabahal,27.711235,85.312934,Central Region
1,Hiranya Varna Mahavihar (Golden Temple),Buddhist Temple,Kwabahal,27.675248,85.324419,Central Region
6,Pashupatinath Temple,Temple,Pashupatinath Rd.,27.709101,85.348620,Central Region
8,Taleju Temple,Temple,Makhan Tole,27.712171,85.311342,Central Region
16,Pachali Bhairav Temple,Temple,Sanepa,27.703415,85.304869,Central Region


## **Search for Restaurants**

In [49]:
# search for Restaurants
search_query = 'Restaurant'
radius = 20000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [29]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [30]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,4c829894d6ebbfb7b3234ca4,Thamel,NP,काठमाडौं,नेपाल,NaN,971,"[Thamel, काठमाडौं 44614, नेपाल]","[{'label': 'display', 'lat': 27.71210903970303...",27.712109,85.311125,NaN,44614,Central Region,Yak Restaurant Bar & Lodge,v-1562688712,NaN
1,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,4c73be8cf4d476b0cc2568cf,Chakshibari Marg,NP,काठमाडौं,नेपाल,Thamel,1188,"[Chakshibari Marg (Thamel), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71463352584197...",27.714634,85.310147,NaN,NaN,Central Region,Yin Yang Restaurant,v-1562688712,NaN
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4ed4d667cc216e1a537fcaaa,NaN,NP,NaN,नेपाल,NaN,858,[नेपाल],"[{'label': 'display', 'lat': 27.71409424486691...",27.714094,85.313907,NaN,NaN,NaN,"Tranzit, Woodfire Pizza, Restaurant & Bar",v-1562688712,NaN
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,5226cc3a93cd4ef097a79f2b,"132, Kwobahal,Thamel",NP,काठमाडौं,नेपाल,Next to Hotel Nepalaya,935,"[132, Kwobahal,Thamel (Next to Hotel Nepalaya)...","[{'label': 'display', 'lat': 27.71167154001644...",27.711672,85.311328,NaN,44600,Central Region,Pilgrims 24 Restaurant & Bar ( Formerly feed ...,v-1562688712,79287569
4,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,5b41d00016fa04002c5397b5,"Thamel-29, Narsing chowck",NP,काठमाडौं,नेपाल,NaN,1050,"[Thamel-29, Narsing chowck, काठमाडौं 44600, ने...","[{'label': 'display', 'lat': 27.71344161494465...",27.713442,85.310970,NaN,44600,Central Region,Nomad's Restaurant and Bar,v-1562688712,501845895


## **Clean Restaurant Dataframe**

In [31]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Yak Restaurant Bar & Lodge,Asian Restaurant,Thamel,NP,काठमाडौं,नेपाल,NaN,971,"[Thamel, काठमाडौं 44614, नेपाल]","[{'label': 'display', 'lat': 27.71210903970303...",27.712109,85.311125,NaN,44614,Central Region,4c829894d6ebbfb7b3234ca4
1,Yin Yang Restaurant,Thai Restaurant,Chakshibari Marg,NP,काठमाडौं,नेपाल,Thamel,1188,"[Chakshibari Marg (Thamel), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71463352584197...",27.714634,85.310147,NaN,NaN,Central Region,4c73be8cf4d476b0cc2568cf
2,"Tranzit, Woodfire Pizza, Restaurant & Bar",Pizza Place,NaN,NP,NaN,नेपाल,NaN,858,[नेपाल],"[{'label': 'display', 'lat': 27.71409424486691...",27.714094,85.313907,NaN,NaN,NaN,4ed4d667cc216e1a537fcaaa
3,Pilgrims 24 Restaurant & Bar ( Formerly feed ...,Restaurant,"132, Kwobahal,Thamel",NP,काठमाडौं,नेपाल,Next to Hotel Nepalaya,935,"[132, Kwobahal,Thamel (Next to Hotel Nepalaya)...","[{'label': 'display', 'lat': 27.71167154001644...",27.711672,85.311328,NaN,44600,Central Region,5226cc3a93cd4ef097a79f2b
4,Nomad's Restaurant and Bar,Asian Restaurant,"Thamel-29, Narsing chowck",NP,काठमाडौं,नेपाल,NaN,1050,"[Thamel-29, Narsing chowck, काठमाडौं 44600, ने...","[{'label': 'display', 'lat': 27.71344161494465...",27.713442,85.310970,NaN,44600,Central Region,5b41d00016fa04002c5397b5


In [32]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood','postalCode', 'id'], axis=1)
clean_Restaurant_dataframe2.head()

,name,categories,address,lat,lng,state
0,Yak Restaurant Bar & Lodge,Asian Restaurant,Thamel,27.712109,85.311125,Central Region
1,Yin Yang Restaurant,Thai Restaurant,Chakshibari Marg,27.714634,85.310147,Central Region
2,"Tranzit, Woodfire Pizza, Restaurant & Bar",Pizza Place,NaN,27.714094,85.313907,NaN
3,Pilgrims 24 Restaurant & Bar ( Formerly feed ...,Restaurant,"132, Kwobahal,Thamel",27.711672,85.311328,Central Region
4,Nomad's Restaurant and Bar,Asian Restaurant,"Thamel-29, Narsing chowck",27.713442,85.310970,Central Region


In [33]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,state
0,Yak Restaurant Bar & Lodge,Asian Restaurant,Thamel,27.712109,85.311125,Central Region
1,Yin Yang Restaurant,Thai Restaurant,Chakshibari Marg,27.714634,85.310147,Central Region
3,Pilgrims 24 Restaurant & Bar ( Formerly feed ...,Restaurant,"132, Kwobahal,Thamel",27.711672,85.311328,Central Region
4,Nomad's Restaurant and Bar,Asian Restaurant,"Thamel-29, Narsing chowck",27.713442,85.310970,Central Region
5,Dallé Restaurant,Restaurant,Kashtamandap Rd.,27.710031,85.319937,Central Region


## **Search for Cafeteria**

In [48]:
# search for Cafeteria
search_query = 'Cafe'
radius = 20000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [35]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [36]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4ea4309a30f8b9d5ad12dd10,Amrit Marg,NP,काठमाडौं,नेपाल,Thamel,1033,"[Amrit Marg (Thamel), काठमाडौं 44600, नेपाल]","[{'label': 'display', 'lat': 27.715046, 'lng':...",27.715046,85.312490,Oposite of TIM Office at Manang Plaza,44600,Central Region,Revolution Cafe,v-1562688763
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d859cde7e8ef04d051625be,Thamel,NP,काठमाडौं,नेपाल,NaN,1244,"[Thamel, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71555500419936...",27.715555,85.310185,NaN,NaN,Central Region,The Northfield Cafe and Jesse James Bar,v-1562688763
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5c00c65178782c002cf88376,NaN,NP,काठमाडौं,नेपाल,NaN,1116,"[काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.716141, 'lng':...",27.716141,85.312529,NaN,NaN,Central Region,The Musketeerz Cafe,v-1562688763
3,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,515a8aeae4b04ee4fe4123a1,Naxal,NP,काठमाडौं,नेपाल,Narayanchour,910,"[Naxal (Narayanchour), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71517974841629...",27.715180,85.326025,NaN,NaN,Central Region,Espression: The Cafe,v-1562688763
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4db18ee3fa8ca4b3e9f9f631,Bhat Bhateni,NP,काठमाडौं,नेपाल,NaN,1674,"[Bhat Bhateni, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.72010609039776...",27.720106,85.331453,Baluwatar,NaN,Central Region,Road House Cafe,v-1562688763


## **Clean Cafeteria Dataframe**

In [52]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Revolution Cafe,Coffee Shop,Amrit Marg,NP,काठमाडौं,नेपाल,Thamel,1033,"[Amrit Marg (Thamel), काठमाडौं 44600, नेपाल]","[{'label': 'display', 'lat': 27.715046, 'lng':...",27.715046,85.312490,Oposite of TIM Office at Manang Plaza,44600,Central Region,4ea4309a30f8b9d5ad12dd10
1,The Northfield Cafe and Jesse James Bar,Café,Thamel,NP,काठमाडौं,नेपाल,NaN,1244,"[Thamel, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71555500419936...",27.715555,85.310185,NaN,NaN,Central Region,4d859cde7e8ef04d051625be
2,The Musketeerz Cafe,Coffee Shop,NaN,NP,काठमाडौं,नेपाल,NaN,1116,"[काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.716141, 'lng':...",27.716141,85.312529,NaN,NaN,Central Region,5c00c65178782c002cf88376
3,Espression: The Cafe,Café,Naxal,NP,काठमाडौं,नेपाल,Narayanchour,910,"[Naxal (Narayanchour), काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.71517974841629...",27.715180,85.326025,NaN,NaN,Central Region,515a8aeae4b04ee4fe4123a1
4,Road House Cafe,Café,Bhat Bhateni,NP,काठमाडौं,नेपाल,NaN,1674,"[Bhat Bhateni, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.72010609039776...",27.720106,85.331453,Baluwatar,NaN,Central Region,4db18ee3fa8ca4b3e9f9f631


In [53]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'postalCode', 'neighborhood', 'id'], axis=1)
clean_Cafeteria_dataframe2.head()

,name,categories,address,lat,lng,state
0,Revolution Cafe,Coffee Shop,Amrit Marg,27.715046,85.312490,Central Region
1,The Northfield Cafe and Jesse James Bar,Café,Thamel,27.715555,85.310185,Central Region
2,The Musketeerz Cafe,Coffee Shop,NaN,27.716141,85.312529,Central Region
3,Espression: The Cafe,Café,Naxal,27.715180,85.326025,Central Region
4,Road House Cafe,Café,Bhat Bhateni,27.720106,85.331453,Central Region


In [54]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria.head()

,name,categories,address,lat,lng,state
0,Revolution Cafe,Coffee Shop,Amrit Marg,27.715046,85.312490,Central Region
1,The Northfield Cafe and Jesse James Bar,Café,Thamel,27.715555,85.310185,Central Region
3,Espression: The Cafe,Café,Naxal,27.715180,85.326025,Central Region
4,Road House Cafe,Café,Bhat Bhateni,27.720106,85.331453,Central Region
5,Cafe Mondo Bizarro,Restaurant,Freak Street,27.703222,85.307922,Central Region


## **Search for Shopping Stores**

In [55]:
# search for Shopping
search_query = 'Mall'
radius = 20000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [56]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [57]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,556ac8d7498e64da5cd5548f,NaN,NP,NaN,नेपाल,NaN,160,[नेपाल],"[{'label': 'display', 'lat': 27.709596, 'lng':...",27.709596,85.318886,NaN,NaN,"The Collective, Rising Mall",v-1562689770
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4c6bb91d0c3ac9b6a78dd238,NaN,NP,NaN,नेपाल,NaN,170,[नेपाल],"[{'label': 'display', 'lat': 27.70887801979128...",27.708878,85.321969,NaN,NaN,China Town Mall,v-1562689770
2,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4c99d148d4b1b1f7348fca35,Tripureshwor,NP,काठमाडौं,नेपाल,NaN,1931,"[Tripureshwor, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.69167187818288...",27.691672,85.317112,NaN,Central Region,Bluebird Mall,v-1562689770
3,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,5956305df4b5252a672602f7,NaN,NP,काठमाडौं,नेपाल,NaN,247,"[काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.710111, 'lng':...",27.710111,85.318224,NaN,Central Region,Madame Rising Mall,v-1562689770
4,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,5291e861498e3ee2d83f0dcb,Kamaladi,NP,NaN,नेपाल,NaN,171,"[Kamaladi, नेपाल]","[{'label': 'display', 'lat': 27.70994931134238...",27.709949,85.319086,NaN,NaN,Rising Mall,v-1562689770


## **Clean Shopping Dataframe**

In [58]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,"The Collective, Rising Mall",Women's Store,NaN,NP,NaN,नेपाल,NaN,160,[नेपाल],"[{'label': 'display', 'lat': 27.709596, 'lng':...",27.709596,85.318886,NaN,NaN,556ac8d7498e64da5cd5548f
1,China Town Mall,Shopping Mall,NaN,NP,NaN,नेपाल,NaN,170,[नेपाल],"[{'label': 'display', 'lat': 27.70887801979128...",27.708878,85.321969,NaN,NaN,4c6bb91d0c3ac9b6a78dd238
2,Bluebird Mall,Department Store,Tripureshwor,NP,काठमाडौं,नेपाल,NaN,1931,"[Tripureshwor, काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.69167187818288...",27.691672,85.317112,NaN,Central Region,4c99d148d4b1b1f7348fca35
3,Madame Rising Mall,Women's Store,NaN,NP,काठमाडौं,नेपाल,NaN,247,"[काठमाडौं, नेपाल]","[{'label': 'display', 'lat': 27.710111, 'lng':...",27.710111,85.318224,NaN,Central Region,5956305df4b5252a672602f7
4,Rising Mall,Shopping Mall,Kamaladi,NP,NaN,नेपाल,NaN,171,"[Kamaladi, नेपाल]","[{'label': 'display', 'lat': 27.70994931134238...",27.709949,85.319086,NaN,NaN,5291e861498e3ee2d83f0dcb


In [59]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                       'crossStreet', 'postalCode' ,'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2.head()

,name,categories,address,lat,lng,state
0,"The Collective, Rising Mall",Women's Store,NaN,27.709596,85.318886,NaN
1,China Town Mall,Shopping Mall,NaN,27.708878,85.321969,NaN
2,Bluebird Mall,Department Store,Tripureshwor,27.691672,85.317112,Central Region
3,Madame Rising Mall,Women's Store,NaN,27.710111,85.318224,Central Region
4,Rising Mall,Shopping Mall,Kamaladi,27.709949,85.319086,NaN


In [60]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping.head()

,name,categories,address,lat,lng,state
1,China Town Mall,Shopping Mall,NaN,27.708878,85.321969,NaN
4,Rising Mall,Shopping Mall,Kamaladi,27.709949,85.319086,NaN
5,Kathmandu Mall,Shopping Mall,Kathmandu,27.701529,85.313348,Central Region
7,Times Square Mall,Shopping Mall,NaN,27.710723,85.317481,NaN
8,Sherpa Mall,Shopping Mall,"DURBAR MARG, KATHMANDU, NEPAL Kathmandu,",27.710692,85.317591,nepal


In [61]:
# delete rows with none values
clean_Shopping_dataframe2 = clean_Shopping_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_Shopping_dataframe2.head()

,name,categories,address,lat,lng,state
2,Bluebird Mall,Department Store,Tripureshwor,27.691672,85.317112,Central Region
5,Kathmandu Mall,Shopping Mall,Kathmandu,27.701529,85.313348,Central Region
6,Sherpa Mall Coffee Express,Coffee Shop,Durbar Marg,27.710735,85.317734,Central Region
8,Sherpa Mall,Shopping Mall,"DURBAR MARG, KATHMANDU, NEPAL Kathmandu,",27.710692,85.317591,nepal
12,Civil Mall,Shopping Mall,Sundhara,27.699399,85.312736,Central Region


## **Generate maps to visualize venues and how they cluster together**

In [62]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
tourist_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(df_hotels['lat'], df_hotels['lng'], 
                                           df_hotels['name'], df_hotels['categories'],
                                              df_hotels['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(tourist_map) 
    
for lat, lng, name, categories, address in zip(df_Cafeteria['lat'], df_Cafeteria['lng'], 
                                           df_Cafeteria['name'], df_Cafeteria['categories'],
                                              df_Cafeteria['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(tourist_map)

for lat, lng, name, categories, address in zip(df_Shopping['lat'], df_Shopping['lng'], 
                                           df_Shopping['name'], df_Shopping['categories'],
                                              df_Shopping['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(tourist_map)  

for lat, lng, name, categories, address in zip(df_temples['lat'], df_temples['lng'], 
                                           df_temples['name'], df_temples['categories'],
                                              df_temples['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(tourist_map)  
    
tourist_map

## Analytical Output
#### The above map ashows that the best place for tourist to live is the place which has the fastest way to get Thamel area and Durbarmarg area because the cluster of places is highly dense in this area.

##### (Look at the screenshot of map on github)